In [55]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm.notebook import tqdm

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [56]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [57]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [58]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [59]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.01) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.99))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [60]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [61]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [62]:
#generate_new_params(2000)

In [63]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [64]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [65]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [66]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    for i, RA in enumerate(risk_aversion):
        if RA > 0.95:
            risk_aversion[i] = risk_aversion[i] - 0.1

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [67]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [68]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [69]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [70]:
probabilities, d_target, g_res = sample_from_csv(100)
A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [71]:
def agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.001
    alpha = [[proba/(1 - min(risk_aversion)) for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [100000 for proba in probabilities]

    j_max = [10 for i in range(community_size)]

    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [72]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [73]:
def gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)

    if solution_type == 'centralized_optimistic':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)
    if solution_type == 'centralized_without_finance':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)
    if solution_type == 'centralized_pessimistic':
        agents = agents_list_pessimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)
    if solution_type == 'risk-neutral':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)
        
    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()
        
        
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [74]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [75]:
connection_matrix_2 = text_to_adj_matrix('../matrices/matrix_2.txt')

In [76]:
def experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='risk-neutral', verbosity = 0):


    total_results= {}
    err_track = []
    for index in tqdm(df_param.index.levels[0]):
        vars, objective, model = gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [77]:
res_RN, df_RN, model_RN = experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res,
            df_param, solution_type='risk-neutral')

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/1999 [00:00<?, ?it/s]

In [79]:
df_RN

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,q_24_19_91,q_24_19_92,q_24_19_93,q_24_19_94,q_24_19_95,q_24_19_96,q_24_19_97,q_24_19_98,q_24_19_99,objective
0,5.286256e-12,3.043739e-02,9.476742e-02,1.584826e-01,2.213211e-01,2.841185e-01,3.469159e-01,0.409206,0.471130,0.533054,...,-9.121591,-9.035185,-8.950740,-8.865609,-8.744200,-8.610558,-8.489512,-8.368376,-8.247228,0.879629
1,5.371662e-12,1.045831e-11,1.106255e-11,2.742267e-11,6.478096e-11,1.737470e-10,8.025364e-10,0.038520,0.091528,0.142542,...,-3.656126,-4.226639,-4.650725,-5.031600,-5.276196,-5.498060,-5.652426,-5.612803,-5.420839,-0.645193
2,6.994571e-12,2.657085e-11,6.784016e-11,3.226012e-02,9.250209e-02,1.483758e-01,2.041685e-01,0.256160,0.303117,0.352124,...,-5.137839,-5.223607,-5.309364,-5.395119,-5.481837,-5.649514,-5.839246,-5.968854,-6.039998,-3.226132
3,2.388909e-12,4.723389e-12,5.050938e-12,1.257297e-11,2.852637e-11,7.205433e-11,2.242492e-10,0.028671,0.076140,0.121317,...,-7.578696,-7.550853,-7.530757,-7.523797,-7.522658,-7.532890,-7.546092,-7.567657,-7.593485,2.602238
4,3.243370e-11,3.052286e-02,9.936995e-02,1.643158e-01,2.281281e-01,2.910493e-01,3.528515e-01,0.414655,0.476056,0.537591,...,-8.214723,-8.183953,-8.153906,-8.124334,-8.113748,-8.104614,-8.085039,-8.072493,-8.015555,0.895023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.389571e-11,1.751542e-10,6.953748e-03,6.056992e-02,1.118348e-01,1.630921e-01,2.143494e-01,0.265607,0.315744,0.364286,...,-4.803345,-4.915194,-5.010448,-5.105696,-5.222868,-5.364930,-5.503356,-5.632006,-5.758615,1.180748
1996,3.333932e-12,4.639012e-11,6.071771e-02,1.226499e-01,1.819524e-01,2.416843e-01,3.016227e-01,0.360080,0.418528,0.477032,...,-10.000000,-9.870437,-9.677394,-9.553496,-9.408151,-9.265109,-9.071908,-8.854807,-8.636520,0.817324
1997,5.467819e-12,3.240274e-11,1.399669e-10,4.286229e-02,9.962502e-02,1.513141e-01,2.018423e-01,0.252017,0.298857,0.345767,...,-3.674976,-3.981372,-4.158403,-4.345285,-4.514253,-4.682180,-4.847215,-5.059444,-5.290689,5.583389
1998,2.532300e-11,2.641020e-02,9.523185e-02,1.576335e-01,2.195914e-01,2.805906e-01,3.416923e-01,0.402036,0.461664,0.520955,...,-3.497377,-3.268599,-3.039841,-2.811079,-2.582344,-2.353606,-2.297702,-2.356144,-2.683267,1.464802


In [80]:
def agent_utility_from_df_row(df_row, agents):
    res = []
    
    for agent in agents:
        agent_obj = 0

        for proba in agent.probabilities_ind:
            D_idx = f'D_{agent.id}_{proba}'
            G_idx = f'G_{agent.id}_{proba}'

            quad_d = agent.a_tilde * (df_row[D_idx] - agent.D_target[proba]) ** 2 - agent.b_tilde 
            quad_g = 0.5 * agent.a * df_row[G_idx] * df_row[G_idx] + agent.b * df_row[G_idx] + agent.d

            lin_q = 0
            
            for agent_2 in agents:
                if agent.connections[agent_2.id]:
                    q_idx = f'q_{agent.id}_{agent_2.id}_{proba}'
                    lin_q += df_row[q_idx] * agent.trading_cost[agent_2.id]
            

            agent_obj += agent.probabilities[proba] * (quad_d + quad_g + lin_q)

        res.append(agent_obj)

    return res

In [81]:
agent_objectives = []
for index, row in tqdm(df_RN.iterrows()):
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)
    agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)
    agent_objectives.append(agent_utility_from_df_row(row, df_param, index))

0it [00:00, ?it/s]

In [82]:
dict_df_param = df_param.to_dict()
for i in range(community_size):
    for trial_idx, trial in enumerate(agent_objectives):
        dict_df_param[f'{i}'].update({(trial_idx, 'objective') : trial[i]})

df_param = pd.DataFrame(dict_df_param).sort_index(level=0)
df_param
    

0         1         2         3         4  \
0    A_tilde        0.717547  0.654177  0.013093  0.877737  0.104077   
     B_tilde        0.321196  0.454466  0.749068  0.460760  0.372843   
     a              0.989779  0.217117  0.606744  0.074443  0.541778   
     b              0.636386  0.172969  0.723784  0.608133  0.861277   
     d              0.484944  0.755008  0.937329  0.727731  0.151528   
...                      ...       ...       ...       ...       ...   
1999 a              0.369826  0.189197  0.529484  0.746706  0.067378   
     b              0.165610  0.284443  0.799789  0.344621  0.655007   
     d              0.160511  0.253647  0.078895  0.003548  0.356696   
     objective     -0.466476  0.014901 -0.371336 -1.082939 -0.241659   
     risk_aversion  0.614504  0.366860  0.442343  0.369069  0.814488   

                           5         6         7         8         9  ...  \
0    A_tilde        0.967468  0.406434  0.921869  0.028222  0.039824  ...   
     B_tilde        0.908785  0.912784  0.468212  0.332271  0.937093  ...   
     a              0.210180  0.136613  0.981449  0.533806  0.062125  ...   
     b              0.718491  0.523857  0.305293  0.481276  0.387807  ...   
     d              0.766490  0.175645  0.453242  0.881070  0.982530  ...   
...                      ...       ...       ...       ...       ...  ...   
1999 a              0.192418  0.881850  0.214001  0.480392  0.216424  ...   
     b              0.095538  0.777456  0.664885  0.335663  0.525495  ...   
     d              0.818701  0.748795  0.930096  0.198560  0.502350  ...   
     objective     -0.224780  0.197325  0.437483 -0.013836  0.086678  ...   
     risk_aversion  0.842747  0.306297  0.392778  0.889301  0.893431  ...   

                          15        16        17        18        19  \
0    A_tilde        0.328909  0.951172  0.267743  0.295853  0.623533   
     B_tilde        0.147292  0.660800  0.544742  0.880750  0.124592   
     a              0.909541  0.129962  0.028962  0.489391  0.102483   
     b              0.920114  0.520913  0.698510  0.752916  0.159282   
     d              0.053421  0.276132  0.421057  0.892414  0.035648   
...                      ...       ...       ...       ...       ...   
1999 a              0.677218  0.049889  0.495318  0.167262  0.208154   
     b              0.020919  0.373464  0.195096  0.755937  0.543812   
     d              0.126103  0.077541  0.000206  0.730584  0.121553   
     objective     -0.952774 -0.327719 -0.585628  0.896152 -0.704672   
     risk_aversion  0.294203  0.536948  0.391047  0.552773  0.035062   

                          20        21        22        23        24  
0    A_tilde        0.973707  0.476848  0.666707  0.155502  0.840464  
     B_tilde        0.279776  0.847677  0.795926  0.096711  0.981173  
     a              0.629521  0.306684  0.514739  0.473038  0.015736  
     b              0.253708  0.064357  0.806582  0.741722  0.066194  
     d              0.915235  0.734328  0.505918  0.427903  0.213077  
...                      ...       ...       ...       ...       ...  
1999 a              0.537937  0.372154  0.950846  0.511756  0.879241  
     b              0.701483  0.355683  0.754106  0.844660  0.259825  
     d              0.173995  0.084644  0.547398  0.256455  0.204323  
     objective     -0.588732  0.375524 -1.263678 -0.106420 -0.151604  
     risk_aversion  0.060813  0.778704  0.201295  0.385945  0.658455  

[14000 rows x 25 columns]

In [83]:
df_RN.to_csv(f'../data/RN_results_{len(df_RN)}.csv')
df_param.to_csv(f'../data/RN_param_{2000}_with_obj.csv')